Control Panel\Hardware and Sound\Devices and Printers

In [1]:
import serial
ser=serial.Serial("COM7",9600,timeout=1)
ser.write(bytes(f"{1},{1}\n", 'utf-8'))

4

In [2]:
import cv2 as cv
from cv2 import aruco
import numpy as np

calib_data_path = "./MultiMatrix.npz"
calib_data = np.load(calib_data_path)
print(calib_data.files)

cam_mat = calib_data["camMatrix"]
dist_coef = calib_data["distCoef"]
r_vectors = calib_data["rVector"]
t_vectors = calib_data["tVector"]

MARKER_SIZE = 8  # centimeters
marker_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
param_markers = aruco.DetectorParameters_create()

['camMatrix', 'distCoef', 'rVector', 'tVector']


In [ ]:
cap = cv.VideoCapture(1)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    marker_corners, marker_IDs, reject = aruco.detectMarkers(
        gray_frame, marker_dict, parameters=param_markers
    )
    if marker_corners:
        rVec, tVec, _ = aruco.estimatePoseSingleMarkers(
            marker_corners, MARKER_SIZE, cam_mat, dist_coef
        )
        total_markers = range(0, marker_IDs.size)
        for ids, corners, i in zip(marker_IDs, marker_corners, total_markers):
            cv.polylines(
                frame, [corners.astype(np.int32)], True, (0, 255, 255), 4, cv.LINE_AA
            )
            corners = corners.reshape(4, 2)
            corners = corners.astype(int)
            top_right = corners[0].ravel()
            top_left = corners[1].ravel()
            bottom_right = corners[2].ravel()
            bottom_left = corners[3].ravel()

            # Since there was mistake in calculating the distance approach point-outed in the Video Tutorial's comment
            # so I have rectified that mistake, I have test that out it increase the accuracy overall.
            # Calculating the distance
            distance = np.sqrt(
                tVec[i][0][2] ** 2 + tVec[i][0][0] ** 2 + tVec[i][0][1] ** 2
            )
            
            # Printing coordinate of distanc
            x=tVec[i][0][0]-(tVec[i][0][2]*0.087+5)
            y = tVec[i][0][2]
            
            if(ids[0]==1):
                coordinate = str(int(x))+","+str(int(y))
                ser.write(bytes(f"{int(x)},{int(y)}\n", 'utf-8'))
                print(coordinate)

            
            # Draw the pose of the marker
            point = cv.drawFrameAxes(frame, cam_mat, dist_coef, rVec[i], tVec[i], 4, 4)
            cv.putText(
                frame,
                f"id: {ids[0]} Dist: {round(distance, 2)}",
                top_right,
                cv.FONT_HERSHEY_PLAIN,
                1.3,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )
            cv.putText(
                frame,
                f"x:{int(x)} y: {int(y)} ",
                bottom_right,
                cv.FONT_HERSHEY_PLAIN,
                1.0,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )
            # print(ids, "  ", corners)
    cv.imshow("frame", frame)
    key = cv.waitKey(1)
    if key == ord("q"):
        break
cap.release()
cv.destroyAllWindows()

10,57
10,58
9,59
2,60
2,60
1,61
1,61
1,62
1,62
1,62
1,62
1,62
1,63
1,64
1,65
1,66
1,66
2,67
2,69
2,69
2,70
2,71
2,71
2,71
2,71
2,72
2,72
3,71
3,71
3,72
3,73
3,72
3,72
3,73
3,73
3,72
3,72
3,72
3,72
3,72
3,73
3,73
3,73
3,73
3,73
3,73
3,72
3,72
3,72
3,72
4,72
4,72
4,72
4,71
4,71
4,71
4,71
5,71
5,71
5,70
5,70
5,69
6,68
6,67
6,67
6,68
6,67
6,66
6,66
6,66
5,65
5,65
5,65
4,65
4,65
3,65
3,65
2,66
2,66
1,67
1,68
1,68
0,68
0,68
0,68
0,69
0,69
0,69
0,69
0,69
0,69
0,69
0,69
0,69
0,69
0,70
0,70
0,70
0,70
0,71
0,71
0,71
0,72
0,73
0,73
0,73
0,73
0,73
0,73
0,73
0,73
0,73
0,73
0,73
0,73
0,73
0,73
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,72
0,71
0,71
1,70
2,67
3,63
5,59
6,55
6,55
17,102
16,100
14,97
2,87
2,86
2,87
2,87
2,87
2,88
2,88
2,88
2,89
2,90
2,90
2,90
2,91
3,91
3,91
3,91
3,91
3,91
4,91
4,92
4,91
4,91
5,92
6,91
6,90
8,91
8,91
10,90
12,92
-30,293
-5,345
-5,327
-5,327
-5,328
-4,324
-4,334
9,390
11,387
-27,345
-34,325
-34,322
-34,312
-34,308
-34,303
-34,299
-34,301

-10,153
-10,153
-10,154
-10,154
-10,155
-11,155
-11,155
-11,155
-11,154
-11,154
-11,156
-11,157
-11,156
-11,155
-10,155
-10,155
-10,155
-10,157
-9,157
-9,159
-9,156
-8,158
-8,158
-8,159
-8,158
-7,159
-7,158
-7,157
-7,157
-7,158
-7,158
-7,160
-6,160
-6,158
-6,158
-6,160
-6,159
-6,159
-6,161
-6,161
-5,159
-5,159
-5,160
-5,162
-5,160
-5,160
-5,160
-5,160
-5,158
-5,158
-5,160
-5,156
-5,156
-5,157
-5,157
-5,158
-5,157
-5,156
-5,157
-5,157
-5,156
-5,155
-5,156
-5,156
-5,157
-5,156
-5,156
-5,158
-5,157
-5,157
-5,157
-5,158
-5,156
-5,158
-5,157
-5,159
-5,158
-5,158
-5,157
-5,159
-5,157
-5,160
-5,159
-5,159
-5,160
-6,158
-6,158
-5,160
-5,160
-5,160
-5,160
-5,161
-5,160
-5,160
-5,159
-5,158
-5,158
-5,160
-5,162
-5,162
-5,160
-5,159
-5,160
-5,162
-5,162
-5,160
-5,160
-5,160
-5,159
-5,158
-5,158
-5,161
-5,161
-4,159
-4,160
-4,160
-3,160
-3,157
-3,159
-3,159
-3,158
-3,160
-3,159
-3,159
-3,159
-3,158
-3,160
-3,162
-3,160
-3,158
-3,159
-3,160
-3,160
-4,159
-4,160
-4,160
-4,160
-4,161
-4,161
-4,159
-3

-1,304
-2,304
-3,302
-2,293
-2,293
0,296
-1,288
-1,286
-2,278
-2,278
-2,278
-2,278
-2,278
-1,282
7,263
7,263
8,261
7,250
7,244
9,252
9,252
11,252
11,252
11,252
11,252
12,239
23,225
23,225
25,223
25,223
26,226
26,220
30,219
30,219
31,218
30,210
31,207
34,213
34,201
35,201
36,202
36,203
35,200
36,193
36,192
36,192
36,192
36,192
36,192
36,196
30,192
30,192
27,205
-5,215
-8,209
-15,213
-18,211
-20,211
-20,211
-22,215
-26,219
-28,219
-27,223
-27,223
-27,222
-27,222
-27,222
-28,232
-28,228
-28,228
-28,232
-29,234
-28,232
-26,234
-24,225
-24,225
-24,225
-23,224
-23,227
-23,228
-22,227
-22,230
-22,230
-21,226
-23,245
-23,245
-21,235
-18,242
-17,233
-17,237
-16,226
-16,226
-16,229
-14,226
-12,219
-11,213
-10,207
-10,207
-9,206
-8,201
-4,198
-4,198
-3,193
-2,190
-2,185
-1,181
-1,181
0,174
0,174
0,169
0,161
0,156
0,162
1,153
2,147
2,147
3,138
5,134
7,131
5,85
5,87
5,88
5,88
4,89
4,90
4,91
4,93
3,94
3,92
3,92
4,90
4,88
4,86
4,84
4,84
4,79
-17,114
-17,118
-18,116
-18,118
-19,120
-19,120
-19,119
-19

-3,49
-3,52
-3,54
-3,57
-3,58
-3,60
-4,63
-4,64
-4,61
-5,60
-5,58
-5,56
-4,52
-3,50
-3,50
-3,49
-3,49
-2,45
-2,44
-2,44
-2,42
-2,40
-2,40
-2,37
-2,37
-2,36
-2,35
-2,34
-2,33
-2,32
-2,31
-2,31
-2,31
-2,30
-2,30
-2,29
-2,29
-3,37
-3,37
-4,49
-4,57
-4,57
-4,57
-4,63
-5,74
-6,80
-6,82
-7,81
-7,79
38,532
38,532
38,532
25,432
25,409
27,420
27,420
28,381
29,373
26,364
26,364
23,345
22,343
21,331
20,316
19,312
17,308
16,296
16,296
14,279
13,273
12,255
12,243
12,223
55,312
55,312
54,315
53,316
55,328
59,353
59,353
53,355
54,382
54,382
53,386
53,386
48,369
47,363
14,315
-10,324
-16,323
-17,337
-17,334
-17,333
-17,336
-17,348
-14,339
17,374
19,374
21,374
25,383
26,383
26,382
26,382
23,404
8,427
8,427
6,415
8,437
8,437
8,437
-2,329
-4,343
-4,343
-5,353
-6,343
-3,333
-3,333
-10,326
-11,327
-12,344
-7,298
-7,298
21,323
23,331
25,341
28,362
28,357
26,379
27,399
26,394
27,395
21,389
23,407
23,407
15,344
18,359
18,347
13,382
13,382
12,375
20,387
73,367
71,353
74,363
75,363
75,363
82,389
74,394
72,402
7

1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,135
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
2,133
2,133
2,132
2,132
2,132
1,132
1,132
1,132
1,133
1,133
1,133
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,132
1,131
1,131
1,131
0,130
0,128
0,129
0,128
0,129
0,129
0,128
0,128
0,128
0,128
0,127
1,128
1,126
1,126
1,126
1,126
1,125
1,125
1,126
1,125
1,126
1,125
2,126
2,126
1,125
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,124
1,125
1,125
1,125
1,126
1,127
1,127
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,125
1,125
1,126
1,126
1,126
1,126
1,126
1,126
1,126
1,127
1,127
1,127
1,126
1,126
1,126
1,125
1,126
1,126
1,126
1,126
1,12

-13,156
-13,159
-13,159
-13,158
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,157
-13,157
-13,157
-13,157
-13,157
-13,157
-13,157
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,157
-13,157
-13,157
-13,157
-13,157
-13,157
-13,157
-13,159
-13,159
-13,159
-13,158
-13,159
-13,158
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-13,159
-12,158
-12,158
-12,158
-11,159
-11,159
-11,157
-11,156
-10,161
-9,157
-9,157
-6,158
-5,158
-5,158
-3,160
-2,161
-1,164
-1,164
-1,164
0,166
0,167
1,174


In [ ]:
cap.release()
cv.destroyAllWindows()